# AWS SageMaker Data Lake and Feature Store for Time Series Forecasting

This notebook creates:
1. **Data Lake** - S3 buckets + Athena tables for raw aggregated time series data
2. **Feature Store** - SageMaker Feature Store for engineered features

This supports the demand forecasting models built in Modeling.ipynb

## Part 1: Data Lake Setup

Initialize SageMaker SDK and configure AWS resources.

In [ ]:
import boto3
import sagemaker
import pandas as pd
import numpy as np
from datetime import datetime
import time

# Initialize SageMaker session
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
s3_client = boto3.client('s3', region_name=region)
athena_client = boto3.client('athena', region_name=region)

print(f"SageMaker Role: {role}")
print(f"Default S3 Bucket: {bucket}")
print(f"Region: {region}")

### Load and Prepare Data
Load dataset.csv and create aggregated + engineered feature tables.

In [ ]:
# Load raw data
df_raw = pd.read_csv('dataset.csv')
print(f"Raw data shape: {df_raw.shape}")
print(f"Raw data columns: {df_raw.columns.tolist()}")

# Aggregate data by timestamp (sum across all locations)
df_raw['timestamp'] = pd.to_datetime(df_raw['timestamp'])
df_agg = df_raw.groupby('timestamp')['value'].sum().reset_index()
df_agg.columns = ['ds', 'y']
df_agg = df_agg.sort_values('ds').reset_index(drop=True)
df_agg['event_time'] = df_agg['ds']  # For SageMaker Feature Store

print(f"\nAggregated data shape: {df_agg.shape}")
print(f"Date range: {df_agg['ds'].min()} to {df_agg['ds'].max()}")
print(df_agg.head())

In [ ]:
# Create engineered features
def create_features(df):
    df = df.copy()
    df['month']         = df['ds'].dt.month
    df['quarter']       = df['ds'].dt.quarter
    df['year']          = df['ds'].dt.year
    df['month_sin']     = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos']     = np.cos(2 * np.pi * df['month'] / 12)
    df['trend']         = np.arange(len(df))

    for lag in [1, 2, 3, 6, 12]:
        df[f'lag_{lag}'] = df['y'].shift(lag)

    df['rolling_mean_3']  = df['y'].shift(1).rolling(3).mean()
    df['rolling_mean_12'] = df['y'].shift(1).rolling(12).mean()
    df['rolling_std_3']   = df['y'].shift(1).rolling(3).std()

    return df

df_feat = create_features(df_agg)
df_feat = df_feat.dropna().reset_index(drop=True)
df_feat['event_time'] = df_feat['ds']  # For SageMaker Feature Store

print(f"Engineered features shape: {df_feat.shape}")
print(f"Feature columns: {[c for c in df_feat.columns if c not in ['ds', 'y']]}")
print(df_feat.head())

### Upload Data to S3 Data Lake

In [ ]:
# Define S3 paths
s3_prefix = 'timeseries-demand-forecasting'
s3_agg_path = f's3://{bucket}/{s3_prefix}/data-lake/aggregated/'
s3_feat_path = f's3://{bucket}/{s3_prefix}/data-lake/features/'
s3_staging_dir = f's3://{bucket}/{s3_prefix}/athena/staging'

# Convert to Parquet for better performance
df_agg.to_parquet('df_agg.parquet', index=False)
df_feat.to_parquet('df_feat.parquet', index=False)

# Upload to S3
sess.upload_data('df_agg.parquet', bucket=bucket, key_prefix=f'{s3_prefix}/data-lake/aggregated/')
sess.upload_data('df_feat.parquet', bucket=bucket, key_prefix=f'{s3_prefix}/data-lake/features/')

print(f"✓ Data uploaded to S3")
print(f"  Aggregated: {s3_agg_path}")
print(f"  Features:   {s3_feat_path}")

### Create Athena Database & Tables

Use Athena to query data in the data lake.

In [ ]:
from pyathena import connect

# Setup Athena connection
database_name = "demand_forecasting"
table_agg = "timeseries_aggregated"
table_feat = "timeseries_features"

conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

# Create database
create_db = f"CREATE DATABASE IF NOT EXISTS {database_name}"
pd.read_sql(create_db, conn)
print(f"✓ Database '{database_name}' created")

# Create aggregated table
create_agg_table = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.{table_agg} (
  ds timestamp,
  y double,
  event_time timestamp
)
STORED AS PARQUET
LOCATION '{s3_agg_path}'
"""
pd.read_sql(create_agg_table, conn)
print(f"✓ Table '{table_agg}' created")

# Create features table
create_feat_table = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.{table_feat} (
  ds timestamp,
  y double,
  event_time timestamp,
  month int,
  quarter int,
  year int,
  month_sin double,
  month_cos double,
  trend int,
  lag_1 double,
  lag_2 double,
  lag_3 double,
  lag_6 double,
  lag_12 double,
  rolling_mean_3 double,
  rolling_mean_12 double,
  rolling_std_3 double
)
STORED AS PARQUET
LOCATION '{s3_feat_path}'
"""
pd.read_sql(create_feat_table, conn)
print(f"✓ Table '{table_feat}' created")

In [ ]:
# Verify tables with queries
query_agg = f"SELECT COUNT(*) as row_count FROM {database_name}.{table_agg}"
df_agg_verify = pd.read_sql(query_agg, conn)
print(f"Aggregated table rows: {df_agg_verify['row_count'].values[0]}")

query_feat = f"SELECT COUNT(*) as row_count FROM {database_name}.{table_feat}"
df_feat_verify = pd.read_sql(query_feat, conn)
print(f"Features table rows: {df_feat_verify['row_count'].values[0]}")

# Sample query
sample_query = f"SELECT ds, y FROM {database_name}.{table_agg} LIMIT 5"
df_sample = pd.read_sql(sample_query, conn)
print(f"\nSample data from aggregated table:")
print(df_sample)

---

## Part 2: SageMaker Feature Store Setup

Create feature groups for aggregated data and engineered features.

In [ ]:
from sagemaker.feature_store.feature_group import FeatureGroup
import time

# Initialize SageMaker Feature Store
print(f"Feature Store initialized for region: {region}")

### Create Feature Group: Aggregated Time Series Data

In [ ]:
# Prepare data for aggregated feature group
fg_agg_name = 'timeseries-aggregated-fg'
df_agg['event_time'] = pd.Timestamp.now().timestamp()

# Create Feature Group object
fg_agg = FeatureGroup(
    name=fg_agg_name,
    sagemaker_session=sess
)

# Load feature definitions from the dataframe
fg_agg.load_feature_definitions(data_frame=df_agg)

print(f"Creating feature group: {fg_agg_name}")

# Delete existing feature group if it exists
try:
    fg_agg.delete()
    print("Found existing feature group. Deleting and waiting 15s for cleanup...")
    time.sleep(15)
except Exception:
    print("No existing feature group found. Proceeding to create.")

# Create feature group with proper parameters
try:
    fg_agg.create(
        s3_uri=f"s3://{bucket}/{s3_prefix}/data-lake/aggregated/",
        record_identifier_name="ds",
        event_time_feature_name="event_time",
        role_arn=role,
        enable_online_store=False
    )
    
    # Wait for creation to complete
    status = fg_agg.describe().get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for feature group creation...")
        time.sleep(5)
        status = fg_agg.describe().get("FeatureGroupStatus")
    
    print(f"✓ Feature group '{fg_agg_name}' created successfully!")
    
    # Ingest data after creation
    print("Ingesting data...")
    fg_agg.ingest(
        data_frame=df_agg,
        max_workers=3,
        wait=True
    )
    print(f"✓ Data ingested to feature group '{fg_agg_name}'")
    
except Exception as e:
    print(f"Error creating feature group: {e}")

### Create Feature Group: Engineered Features

In [ ]:
# Prepare data for engineered features feature group
fg_feat_name = 'timeseries-engineered-features-fg'
df_feat['event_time'] = pd.Timestamp.now().timestamp()

# Create Feature Group object
fg_feat = FeatureGroup(
    name=fg_feat_name,
    sagemaker_session=sess
)

# Load feature definitions from the dataframe
fg_feat.load_feature_definitions(data_frame=df_feat)

print(f"Creating feature group: {fg_feat_name}")

# Delete existing feature group if it exists
try:
    fg_feat.delete()
    print("Found existing feature group. Deleting and waiting 15s for cleanup...")
    time.sleep(15)
except Exception:
    print("No existing feature group found. Proceeding to create.")

# Create feature group with proper parameters
try:
    fg_feat.create(
        s3_uri=f"s3://{bucket}/{s3_prefix}/data-lake/features/",
        record_identifier_name="ds",
        event_time_feature_name="event_time",
        role_arn=role,
        enable_online_store=False
    )
    
    # Wait for creation to complete
    status = fg_feat.describe().get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for feature group creation...")
        time.sleep(5)
        status = fg_feat.describe().get("FeatureGroupStatus")
    
    print(f"✓ Feature group '{fg_feat_name}' created successfully!")
    
    # Ingest data after creation
    print("Ingesting data...")
    fg_feat.ingest(
        data_frame=df_feat,
        max_workers=3,
        wait=True
    )
    print(f"✓ Data ingested to feature group '{fg_feat_name}'")
    
except Exception as e:
    print(f"Error creating feature group: {e}")

### Verify Feature Store Setup

In [ ]:
# Verify feature groups were created and check their status
print("\nVerifying Feature Groups...")
print("=" * 60)

try:
    agg_desc = fg_agg.describe()
    agg_status = agg_desc.get("FeatureGroupStatus")
    print(f"\n{fg_agg_name}:")
    print(f"  Status: {agg_status}")
    print(f"  Record Identifier: ds")
    print(f"  Event Time Feature: event_time")
    print(f"  Total rows in dataframe: {len(df_agg)}")
except Exception as e:
    print(f"  Error describing aggregated feature group: {e}")

try:
    feat_desc = fg_feat.describe()
    feat_status = feat_desc.get("FeatureGroupStatus")
    print(f"\n{fg_feat_name}:")
    print(f"  Status: {feat_status}")
    print(f"  Record Identifier: ds")
    print(f"  Event Time Feature: event_time")
    print(f"  Total rows in dataframe: {len(df_feat)}")
except Exception as e:
    print(f"  Error describing engineered features group: {e}")

print("\n✓ Feature Store setup complete!")

---

### Using Feature Store with Models

Query the feature store to retrieve data for training your forecasting models.

In [ ]:
# Example: Query features from Feature Store using Athena (Offline Store)
print("\nExample: Querying Features from Feature Store (Offline Store)")
print("=" * 60)

try:
    # Get Athena query object from engineered features group
    query = fg_feat.athena_query()
    table_name = query.table_name
    print(f"Athena Table Name: {table_name}")
    
    # Create a SQL query to retrieve sample features
    sql_query = f"""
    SELECT ds, y, month, quarter, year, trend, lag_1, lag_12, rolling_mean_3, rolling_mean_12
    FROM "{table_name}" 
    ORDER BY ds DESC
    LIMIT 10
    """
    
    print(f"\nRunning SQL query on Offline Store...")
    
    # Execute the query
    query.run(
        query_string=sql_query,
        output_location=f"s3://{bucket}/{s3_prefix}/query_results"
    )
    query.wait()
    
    # Get results as dataframe
    df_results = query.as_dataframe()
    
    if df_results.empty:
        print("\nQuery returned 0 rows.")
    else:
        print(f"\n✅ Retrieved {len(df_results)} records from Feature Store:")
        print(df_results.head(10))
        print(f"\n✓ Feature Store is ready for training your forecasting models!")
        
except Exception as e:
    print(f"\nError querying Feature Store: {e}")